In [41]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np


from qbench.schemes.postselection import benchmark_using_postselection, benchmark_certification_using_postselection
from qbench.schemes.direct_sum import benchmark_using_direct_sum, benchmark_certification_using_direct_sum

In [42]:
def state_prep():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit.to_instruction()
    

def u_dag():
    circuit = QuantumCircuit(1)
    circuit.h(0)
    return circuit.to_instruction()

def v0_dag():
    circuit = QuantumCircuit(1)
    circuit.x(0)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    
    return circuit.to_instruction()

def v1_dag():
    circuit = QuantumCircuit(1)
    circuit.x(0)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.x(0)
    return circuit.to_instruction()

def v0_v1_direct_sum_dag():
    circuit = QuantumCircuit(2)
    circuit.p(-np.pi, 0)
    circuit.ry(-2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

In [43]:
from qiskit_aer import StatevectorSimulator

# simulator = AerSimulator()
simulator = StatevectorSimulator()

In [ ]:
result = benchmark_certification_using_postselection(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),
    num_shots_per_measurement=100000
)

result

In [45]:
expected = (1/np.sqrt(2) * np.sqrt(0.95) - 1/np.sqrt(2) * np.sqrt(0.05))**2
expected

0.2820550528229661

In [46]:
result = benchmark_certification_using_direct_sum(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_v1_direct_sum_dag=v0_v1_direct_sum_dag(),
    num_shots_per_measurement=100000
)

result

0.28091